# Read using pandas

In [1]:
import pandas as pd

excel_file = "Pokémon Stats, Learnset etc (v4.0).xlsx"
all_sheets = pd.read_excel(excel_file, sheet_name=None)

In [2]:
print(all_sheets.keys())

dict_keys(['Type Chart', 'Etrian Variants', 'New Moves & Abilities', '#1-151', '#152-251', '#252-386', '4th Gen', 'Paradox'])


In [3]:
# Create a type to store pokemon

class Pokemon_Retrieved:
    def __init__(self, name=None, poke_type=None, abilities=None, evo_lvl=None, learnset=None, hp=None, atk=None, defense=None, spatk=None, spdefense=None, spd=None):
        self.name = name
        self.poke_type = poke_type
        self.abilities = abilities
        self.evo_lvl = evo_lvl
        self.learnset = learnset
        
    def __str__(self):
        return self.name
    
    def __repr__(self):
        return self.name
    
    def get_output_type(self):
        return self.poke_type.split("/")
    
    def get_abilities(self):
        # abilities: { 0: "Overgrow", H: "Chlorophyll" },
        abl = self.abilities.split("/")
        
        ret_str = "{ "
        
        for i, ability in enumerate(abl):
            ret_str += f"{i}: \"{ability}\""
            if i < len(abl) - 1: 
                ret_str += ", "
            
        ret_str += " }"
        return ret_str
    
    def get_internal_name(self):
        name_to_use = self.name
        
        name_to_use = name_to_use.lower() + "odyssey"
        
        # New forms
        name_to_use = name_to_use.replace("⭐ ","")
        
        # Thanks Nidoran
        name_to_use = name_to_use.replace(" (f)", "f")
        
        # Thanks Nidoran pt.2
        name_to_use = name_to_use.replace(" (m)", "m")
        
        # Thanks far and sirfetch'd
        name_to_use = name_to_use.replace("'","")
        
        # Thanks galar forms
        name_to_use = name_to_use.replace(" (galar)", "galar")
        
        # Thanks mr. mime
        name_to_use = name_to_use.replace("mr. ", "mr")
        
        return name_to_use
        

In [4]:
gen1_df = all_sheets['#1-151']

all_pokemon = []

iterable_tuples = gen1_df.dropna(how='all')

rows = list(iterable_tuples.itertuples())

# This gets all pokemon, but not the stats, atleast it should
for i, row in enumerate(rows):
    first_poke_col_num = 1
    second_poke_col_num = 4
    third_poke_col_num = 7
    
    all_poke_col_nums = [first_poke_col_num, second_poke_col_num, third_poke_col_num]
    
    # Loops thru each column that can have a poke in it
    for poke_col_num in all_poke_col_nums:
        poke_col = row[poke_col_num]
        
        # Ensures we only catch pokes that belong here, not yet catching moves or catching mons that point to other mons.
        if(type(poke_col) == str and poke_col.isupper() and poke_col[:2] != "LV" and poke_col[-2] != "➡" and poke_col != "EVOLUTION"):
            # Make the object to be stored
            found_poke = Pokemon_Retrieved(name=poke_col)
            
            # Find the type by going down one row and getting the right col
            found_type = rows[i + 1][poke_col_num + 1]
            found_poke.poke_type = found_type
            
            # Find the abilities by going down two row and getting the right col
            found_abilities = rows[i + 2][poke_col_num + 1]
            found_poke.abilities = found_abilities
            
            # Find the evo lvl by going down three rows and getting the right col
            found_evo_lvl = rows[i + 3][poke_col_num + 1]
            # Will need to take a closer look at this as some of these are item evos, not lvls
            found_poke.evo_lvl = found_evo_lvl
            
            # Find the moves.
            # header is at rows[i + 4], so first move is at rows[i + 5]
            curr_move_offset = 0
            found_learnset = []
            
            # Continue iterating till we reach one of these end conditions. So ugly but functional
            while((i + 5 + curr_move_offset) < len(rows) and type(rows[i + 5 + curr_move_offset][poke_col_num + 1]) == str and rows[i + 5 + curr_move_offset][poke_col_num][:2] == "LV"):
                # Append a tuple where the first value is the level and the second is the move
                found_learnset.append((rows[i + 5 + curr_move_offset][poke_col_num][3:], rows[i + 5 + curr_move_offset][poke_col_num + 1]))
                # Continue itering
                curr_move_offset += 1
                
            found_poke.learnset = found_learnset
            
            all_pokemon.append(found_poke)
    
    
    

In [5]:
all_poke_stats = []

# This should get all the stats
for i, row in enumerate(rows):
    stat_col_num = 10
    hp_col_num = stat_col_num
    atk_col_num = stat_col_num + 1
    def_col_num = stat_col_num + 2
    spatk_col_num = stat_col_num + 3
    spdef_col_num = stat_col_num + 4
    spd_col_num = stat_col_num + 5
    
    stat_col = row[stat_col_num]
    
    # Ensures we only catch pokes that belong here, not yet catching moves or catching mons that point to other mons.
    if(type(stat_col) == str and stat_col.isupper() and stat_col[:2] != "LV" and stat_col[-2] != "➡" and poke_col != "EVOLUTION"):
        # Ensures we don't capture stats, only have to worry about HP cus of how spreadsheets work
        if(stat_col != "HP"):
            # One row down is the header row
            # Two rows down is the Odyssey stat row.
            found_stats = [rows[i + 2][hp_col_num], rows[i + 2][atk_col_num], rows[i + 2][def_col_num], rows[i + 2][spatk_col_num],rows[i + 2][spdef_col_num], rows[i + 2][spd_col_num]]
            
            # Append it to the all_poke_stats array, we merge later
            all_poke_stats.append((stat_col, found_stats))

In [6]:
# Merge pokes with stats

print(len(all_poke_stats))
print(len(all_pokemon))

# Rids all stars

# This is for a quick visual check to ensure these line up, which they do
'''
for x in all_pokemon:
    if x.name[:2] == "⭐ ":
        x.name = x.name[2:]
        
for i, x in enumerate(all_poke_stats):
    if x[0][:2] == "⭐ ":
        all_poke_stats[i] = (x[0][2:], x[1])


print([x.name for x in all_pokemon])
print([x[0] for x in all_poke_stats])
'''

# Set stats
for i in range(len(all_pokemon)):
    curr_poke = all_pokemon[i]
    curr_poke_stats = all_poke_stats[i][1]
    curr_poke.hp = curr_poke_stats[0]
    curr_poke.atk = curr_poke_stats[1]
    curr_poke.defense = curr_poke_stats[2]
    curr_poke.spatk = curr_poke_stats[3]
    curr_poke.spdefense = curr_poke_stats[4]
    curr_poke.spd = curr_poke_stats[5]
    

151
151


In [7]:
print(all_pokemon[0].abilities)
print(all_pokemon[0].get_abilities())

Chlorophyll/Overgrow
{ 0: "Chlorophyll", 1: "Overgrow" }


In [8]:
# Generic entry to pokedex.ts
'''
bulbasaur: {
		num: 1,
		name: "Bulbasaur",
		types: ["Grass", "Poison"],
		genderRatio: { M: 0.875, F: 0.125 },
		baseStats: { hp: 45, atk: 49, def: 49, spa: 65, spd: 65, spe: 45 },
		abilities: { 0: "Overgrow", H: "Chlorophyll" },
		heightm: 0.7,
		weightkg: 6.9,
		color: "Green",
		evos: ["Ivysaur"],
		eggGroups: ["Monster", "Grass"],
	},
'''

# Start here
num = -100
# Increment amt
incr_amt = -1

for i, poke in enumerate(all_pokemon):
    # Print internal name, adding -odyssey to avoid dupes with OG mons that may be modified.
    print(f"{poke.get_internal_name()}: {{")
    # Print num
    print(f"\tnum: {num + (incr_amt * i)},")
    # Print name
    print(f"\tname: \"{poke.name.capitalize()}-Odyssey\",")
    # Print types:
    print(f"\ttypes: {poke.get_output_type()},")
    # Print gender ratio. Useful probably, but hard to get definitely, not in spreadsheet dex.
    print(f"\tgenderRatio: {{ M: 0.5, F: 0.5}},")
    # Print base stats.
    print(f"\tbaseStats: {{ hp: {poke.hp}, atk: {poke.atk}, def: {poke.defense}, spa: {poke.spatk}, spd: {poke.spdefense}, spe: {poke.spd}}},")
    # Print abilties.
    print(f"\tabilities: {poke.get_abilities()},")
    # Print height. Useful probably, but hard to get definitely, not in spreadsheet dex.
    print(f"\theightm: 1.0,")
    # Print weight. Useful probably, but hard to get definitely, not in spreadsheet dex.
    print("\tweightkg: 1.0,")
    # Print color. Useful probably, but hard to get definitely, not in spreadsheet dex.
    print("\tcolor: \"Green\",")
    # Print evos. Useful probably, most likely makes eviolite work. I can maybe substite a specific mon in just to make eviolite work for now, but it wont be the correct one.
    # print("\tevos: []")
    # Print egg groups. Useful probably, but hard to get definitely, not in spreadsheet dex.
    print("\teggGroups: [\"Undiscovered\"],")
    # Print tiers if you want
    print("},")
    

bulbasaurodyssey: {
	num: -100,
	name: "Bulbasaur-Odyssey",
	types: ['Grass', 'Poison'],
	genderRatio: { M: 0.5, F: 0.5},
	baseStats: { hp: 45, atk: 49, def: 49, spa: 65, spd: 65, spe: 45},
	abilities: { 0: "Chlorophyll", 1: "Overgrow" },
	heightm: 1.0,
	weightkg: 1.0,
	color: "Green",
	eggGroups: ["Undiscovered"],
},
ivysaurodyssey: {
	num: -101,
	name: "Ivysaur-Odyssey",
	types: ['Grass', 'Poison'],
	genderRatio: { M: 0.5, F: 0.5},
	baseStats: { hp: 60, atk: 62, def: 63, spa: 80, spd: 80, spe: 60},
	abilities: { 0: "Chlorophyll", 1: "Overgrow" },
	heightm: 1.0,
	weightkg: 1.0,
	color: "Green",
	eggGroups: ["Undiscovered"],
},
venusaurodyssey: {
	num: -102,
	name: "Venusaur-Odyssey",
	types: ['Grass', 'Poison'],
	genderRatio: { M: 0.5, F: 0.5},
	baseStats: { hp: 80, atk: 82, def: 83, spa: 100, spd: 100, spe: 80},
	abilities: { 0: "Chlorophyll", 1: "Overgrow" },
	heightm: 1.0,
	weightkg: 1.0,
	color: "Green",
	eggGroups: ["Undiscovered"],
},
charmanderodyssey: {
	num: -103,
	name: "Cha

In [10]:
# Generic entry to pokedex.js

'''
bulbasaur: {
		num: 1,
		name: "Bulbasaur",
		types: ["Grass", "Poison"],
		genderRatio: { M: 0.875, F: 0.125 },
		baseStats: { hp: 45, atk: 49, def: 49, spa: 65, spd: 65, spe: 45 },
		abilities: { 0: "Overgrow", H: "Chlorophyll" },
		heightm: 0.7,
		weightkg: 6.9,
		color: "Green",
		evos: ["Ivysaur"],
		eggGroups: ["Monster", "Grass"],
		tier: "LC",
	},
'''

# Start here
num = -100
# Increment amt
incr_amt = -1

for i, poke in enumerate(all_pokemon):
    # Print internal name, adding -odyssey to avoid dupes with OG mons that may be modified.
    print(f"{poke.get_internal_name()}: {{")
    # Print num
    print(f"\tnum: {num + (incr_amt * i)},")
    # Print name
    print(f"\tname: \"{poke.name.capitalize()}-Odyssey\",")
    # Print types:
    print(f"\ttypes: {poke.get_output_type()},")
    # Print gender ratio. Useful probably, but hard to get definitely, not in spreadsheet dex.
    print(f"\tgenderRatio: {{ M: 0.5, F: 0.5}},")
    # Print base stats.
    print(f"\tbaseStats: {{ hp: {poke.hp}, atk: {poke.atk}, def: {poke.defense}, spa: {poke.spatk}, spd: {poke.spdefense}, spe: {poke.spd}}},")
    # Print abilties.
    print(f"\tabilities: {poke.get_abilities()},")
    # Print height. Useful probably, but hard to get definitely, not in spreadsheet dex.
    print(f"\theightm: 1.0,")
    # Print weight. Useful probably, but hard to get definitely, not in spreadsheet dex.
    print("\tweightkg: 1.0,")
    # Print color. Useful probably, but hard to get definitely, not in spreadsheet dex.
    print("\tcolor: \"Green\",")
    # Print evos. Useful probably, most likely makes eviolite work. I can maybe substite a specific mon in just to make eviolite work for now, but it wont be the correct one.
    # print("\tevos: []")
    # Print egg groups. Useful probably, but hard to get definitely, not in spreadsheet dex.
    print("\teggGroups: [\"Undiscovered\"],")
    # Print tiers if you want
    print("\ttier: \"OU\",")
    print("},")

bulbasaurodyssey: {
	num: -100,
	name: "Bulbasaur-Odyssey",
	types: ['Grass', 'Poison'],
	genderRatio: { M: 0.5, F: 0.5},
	baseStats: { hp: 45, atk: 49, def: 49, spa: 65, spd: 65, spe: 45},
	abilities: { 0: "Chlorophyll", 1: "Overgrow" },
	heightm: 1.0,
	weightkg: 1.0,
	color: "Green",
	eggGroups: ["Undiscovered"],
	tier: "OU",
},
ivysaurodyssey: {
	num: -101,
	name: "Ivysaur-Odyssey",
	types: ['Grass', 'Poison'],
	genderRatio: { M: 0.5, F: 0.5},
	baseStats: { hp: 60, atk: 62, def: 63, spa: 80, spd: 80, spe: 60},
	abilities: { 0: "Chlorophyll", 1: "Overgrow" },
	heightm: 1.0,
	weightkg: 1.0,
	color: "Green",
	eggGroups: ["Undiscovered"],
	tier: "OU",
},
venusaurodyssey: {
	num: -102,
	name: "Venusaur-Odyssey",
	types: ['Grass', 'Poison'],
	genderRatio: { M: 0.5, F: 0.5},
	baseStats: { hp: 80, atk: 82, def: 83, spa: 100, spd: 100, spe: 80},
	abilities: { 0: "Chlorophyll", 1: "Overgrow" },
	heightm: 1.0,
	weightkg: 1.0,
	color: "Green",
	eggGroups: ["Undiscovered"],
	tier: "OU",
},
charm

# Resulting values were pasted directly into pokedex.ts and pokedex.js
## To remove them, search for num: -100 to see where they start.